In [4]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import tool
import pylab
import os

%matplotlib inline
pylab.rcParams['figure.figsize'] = (15.0,7.0) #调整显示大小
tool.setGrid(1,2)

class Detection:
    def __init__(self,xml,color):
        self.xml = xml
        self.color = color
        self.facesCascadc = cv.CascadeClassifier(xml)
    def haar_execute(self,gray):
        blocks = self.facesCascadc.detectMultiScale(
            gray,
            scaleFactor = 1.15,
            minNeighbors = 5,
            minSize = (5,5)
        )
        self.blocks = blocks
        return blocks
    def draw_blocks(self,src,copy=False):
        dis = src
        if(copy):
            dis = np.array(src)
        for (x,y,w,h) in self.blocks:
            cv.rectangle(dis,(x,y),(x+w,y+h),self.color,2)
        return dis

# https://github.com/opencv/opencv/tree/master/data/haarcascades
# faceDet = Detection('./tempdoc/haarcascade_profileface.xml',(255,50,50))
faceDet = Detection('./tempdoc/haarcascade_frontalface_default.xml',(255,150,150))
eyeDet = Detection('./tempdoc/haarcascade_eye.xml',(155,150,250))
# leyeDet = Detection('./tempdoc/haarcascade_lefteye_2splits.xml',(155,250,250))
# reyeDet = Detection('./tempdoc/haarcascade_righteye_2splits.xml',(255,150,250))
# smileDet = Detection('./tempdoc/haarcascade_smile.xml',(50,50,250))

def detAdd2Img(frame):
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    faceDet.haar_execute(gray)
    dis = faceDet.draw_blocks(frame)
    eyeDet.haar_execute(gray)
    dis = eyeDet.draw_blocks(dis)
#     leyeDet.haar_execute(gray)
#     dis = leyeDet.draw_blocks(dis)
#     reyeDet.haar_execute(gray)
#     dis = reyeDet.draw_blocks(dis)
#     smileDet.haar_execute(gray)
#     dis = smileDet.draw_blocks(dis)
#     cv.imshow('frame',frame)
#     cv.imshow('gray',gray)
    return dis

print(True)

True


In [1]:
!arp -a


接口: 192.168.1.189 --- 0x3
  Internet 地址         物理地址              类型
  192.168.1.1           58-6a-b1-ea-ed-66     动态        
  192.168.1.7           78-44-fd-82-2d-fe     动态        
  192.168.1.187         00-1e-67-b0-9f-c2     动态        
  192.168.1.205         b8-97-5a-42-03-63     动态        
  192.168.1.242         fc-aa-14-40-06-63     动态        
  192.168.1.254         00-e2-69-08-00-12     动态        
  192.168.1.255         ff-ff-ff-ff-ff-ff     静态        
  224.0.0.2             01-00-5e-00-00-02     静态        
  224.0.0.22            01-00-5e-00-00-16     静态        
  224.0.0.251           01-00-5e-00-00-fb     静态        
  224.0.0.252           01-00-5e-00-00-fc     静态        
  230.0.0.1             01-00-5e-00-00-01     静态        
  239.255.255.250       01-00-5e-7f-ff-fa     静态        
  255.255.255.255       ff-ff-ff-ff-ff-ff     静态        


In [37]:
# https://anaconda.org/mrinaljain17/ffmpeg-python
# https://kkroening.github.io/ffmpeg-python/
# https://github.com/kkroening/ffmpeg-python
## conda 的 ffmpeg-python 版本不支持 所以用pip安装了

import ffmpeg
# !conda list
# !pip list
width,height = 640,480
process1 = (
    ffmpeg
    .input("rtsp://192.168.31.7:554/live",rtsp_transport='udp')
    .output('pipe:', format='rawvideo', pix_fmt='bgr24')
    .run_async(pipe_stdout=True)
)


while True:
    in_bytes = process1.stdout.read(width * height * 3)
    if not in_bytes:
        print('nodata')
        break
    in_frame = (
        np
        .frombuffer(in_bytes, np.uint8)
        .reshape([height, width, 3])
    )
#     det = cv.pyrDown(in_frame)

    det = in_frame
#     det = in_frame[np.arange(0,height-1,2),:]    
#     det = det[:,np.arange(0,width-1,2)]
#     det = detAdd2Img(det)
    cv.imshow("rtsp_udp",det)
#     cv.imwrite("rtsp_udp.jpg",in_frame)
    key = cv.waitKey(12)  # 延迟
    if key & 0xFF ==27:
        break

cv.destroyAllWindows()
# process1.wait()
#close ??
print('end')

end
